In [5]:
import pandas as pd
df= pd.read_csv("superstore_data_ready_to_model.csv")

In [3]:
df.head()

,Year_Birth,Education,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,...,date,year,Marital_Status_Absurd,Marital_Status_Alone,Marital_Status_Divorced,Marital_Status_Married,Marital_Status_Single,Marital_Status_Together,Marital_Status_Widow,Marital_Status_YOLO
0,0.098441,-0.871658,1.294840,-0.823039,-0.928972,-1.693488,-0.344228,1.951592,0.945465,1.340203,...,0.045129,1.417393,-0.030056,-0.036819,2.924333,-0.794110,-0.519533,-0.590553,-0.188452,-0.030056
1,-0.652633,-0.871658,0.192461,-0.823039,-0.928972,-1.693488,0.471186,-0.536787,-0.459326,-0.559697,...,-0.068129,1.417393,-0.030056,-0.036819,-0.341958,-0.794110,1.924807,-0.590553,-0.188452,-0.030056
2,-0.902990,-0.871658,0.596794,-0.823039,0.909066,-1.693488,-0.507311,-0.385976,-0.481624,-0.413550,...,-0.294645,1.417393,-0.030056,-0.036819,-0.341958,1.259271,-0.519533,-0.590553,-0.188452,-0.030056
3,-0.151917,-0.871658,-0.785669,1.039938,0.909066,-1.693488,-0.874989,-0.662463,-0.740284,-0.687574,...,-1.200710,1.417393,-0.030056,-0.036819,-0.341958,-0.794110,-0.519533,1.693329,-0.188452,-0.030056
4,1.684040,-0.871658,-1.222743,1.039938,-0.928972,-1.693488,-0.886850,-0.260301,-0.637712,-0.486623,...,-1.313968,1.417393,-0.030056,-0.036819,-0.341958,-0.794110,1.924807,-0.590553,-0.188452,-0.030056


In [3]:
df.columns

Index(['Year_Birth', 'Education', 'Income', 'Kidhome', 'Teenhome', 'Recency',
       'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts',
       'MntSweetProducts', 'MntGoldProds', 'NumDealsPurchases',
       'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases',
       'NumWebVisitsMonth', 'Response', 'Complain', 'month', 'date', 'year',
       'Marital_Status_Absurd', 'Marital_Status_Alone',
       'Marital_Status_Divorced', 'Marital_Status_Married',
       'Marital_Status_Single', 'Marital_Status_Together',
       'Marital_Status_Widow', 'Marital_Status_YOLO'],
      dtype='object')

In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Assuming df_encoded is your DataFrame and 'Response' is the target column
# Ensure there are no missing values in the target column
df = df.dropna(subset=['Response'])

# Convert 'Response' to integer if it is not already
df['Response'] = df['Response'].astype(int)

# Split the data into features and target
X = df.drop('Response', axis=1)
y = df['Response']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the models and their parameter grids
models = {
    'RandomForest': {
        'model': RandomForestClassifier(),
        'params': {
            'n_estimators': [50, 100, 200],
            'max_depth': [None, 10, 20, 30],
            'min_samples_split': [2, 5, 10]
        }
    },
    'SVC': {
        'model': SVC(),
        'params': {
            'C': [0.1, 1, 10, 100],
            'gamma': [1, 0.1, 0.01, 0.001],
            'kernel': ['rbf', 'linear']
        }
    },
    'LogisticRegression': {
        'model': LogisticRegression(max_iter=1000),
        'params': {
            'C': [0.1, 1, 10, 100],
            'solver': ['liblinear', 'lbfgs']
        }
    }
}

# Perform GridSearchCV for each model
best_models = {}
for model_name, model_info in models.items():
    print(f"Processing {model_name}...")
    clf = GridSearchCV(model_info['model'], model_info['params'], cv=5, n_jobs=-1, verbose=2, error_score='raise')
    try:
        clf.fit(X_train, y_train)
        best_models[model_name] = clf.best_estimator_
        print(f"Best parameters for {model_name}: {clf.best_params_}")
    except Exception as e:
        print(f"Error occurred for {model_name}: {e}")

# Evaluate the best models on the test set
for model_name, model in best_models.items():
    y_pred = model.predict(X_test)
    print(f"\nClassification report for {model_name}:\n")
    print(classification_report(y_test, y_pred))


Processing RandomForest...
Fitting 5 folds for each of 36 candidates, totalling 180 fits
Best parameters for RandomForest: {'max_depth': None, 'min_samples_split': 5, 'n_estimators': 100}
Processing SVC...
Fitting 5 folds for each of 32 candidates, totalling 160 fits
Best parameters for SVC: {'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}
Processing LogisticRegression...
Fitting 5 folds for each of 8 candidates, totalling 40 fits
Best parameters for LogisticRegression: {'C': 1, 'solver': 'liblinear'}

Classification report for RandomForest:

              precision    recall  f1-score   support

           0       0.89      0.98      0.93       379
           2       0.71      0.31      0.43        65

    accuracy                           0.88       444
   macro avg       0.80      0.64      0.68       444
weighted avg       0.87      0.88      0.86       444


Classification report for SVC:

              precision    recall  f1-score   support

           0       0.89      0.97      0.93

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, r2_score

# Assuming df_encoded is your DataFrame and 'Response' is the target column
# Ensure there are no missing values in the target column
df = df.dropna(subset=['Response'])

# Convert 'Response' to integer if it is not already
df['Response'] = df['Response'].astype(int)

# Split the data into features and target
X = df.drop('Response', axis=1)
y = df['Response']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the models and their parameter grids
models = {
    'RandomForest': {
        'model': RandomForestClassifier(),
        'params': {
            'n_estimators': [50, 100, 200],
            'max_depth': [None, 10, 20, 30],
            'min_samples_split': [2, 5, 10]
        }
    },
    'SVC': {
        'model': SVC(),
        'params': {
            'C': [0.1, 1, 10, 100],
            'gamma': [1, 0.1, 0.01, 0.001],
            'kernel': ['rbf', 'linear']
        }
    },
    'LogisticRegression': {
        'model': LogisticRegression(max_iter=1000),
        'params': {
            'C': [0.1, 1, 10, 100],
            'solver': ['liblinear', 'lbfgs']
        }
    }
}

# Perform GridSearchCV for each model
best_models = {}
for model_name, model_info in models.items():
    print(f"Processing {model_name}...")
    clf = GridSearchCV(model_info['model'], model_info['params'], cv=5, n_jobs=-1, verbose=2, error_score='raise')
    try:
        clf.fit(X_train, y_train)
        best_models[model_name] = clf.best_estimator_
        print(f"Best parameters for {model_name}: {clf.best_params_}")
    except Exception as e:
        print(f"Error occurred for {model_name}: {e}")

# Evaluate the best models on the test set
for model_name, model in best_models.items():
    y_pred = model.predict(X_test)
    print(f"\nClassification report for {model_name}:\n")
    print(classification_report(y_test, y_pred))

    # Compute R-squared value
    r2 = r2_score(y_test, y_pred)
    print(f"R-squared for {model_name}: {r2:.4f}")


Processing RandomForest...
Fitting 5 folds for each of 36 candidates, totalling 180 fits
Best parameters for RandomForest: {'max_depth': 20, 'min_samples_split': 10, 'n_estimators': 50}
Processing SVC...
Fitting 5 folds for each of 32 candidates, totalling 160 fits
Best parameters for SVC: {'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}
Processing LogisticRegression...
Fitting 5 folds for each of 8 candidates, totalling 40 fits
Best parameters for LogisticRegression: {'C': 1, 'solver': 'liblinear'}

Classification report for RandomForest:

              precision    recall  f1-score   support

           0       0.88      0.98      0.93       379
           2       0.71      0.23      0.35        65

    accuracy                           0.87       444
   macro avg       0.80      0.61      0.64       444
weighted avg       0.86      0.87      0.85       444

R-squared for RandomForest: -0.0093

Classification report for SVC:

              precision    recall  f1-score   support

         